# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [110]:
##### Your Code Here #####

import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD, Adam, Nadam
from keras.wrappers.scikit_learn import KerasClassifier

import pandas as pd
import numpy as np
import category_encoders as ce


from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold, KFold, GridSearchCV 
              
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# fix random seed for reproducibility
np.random.seed(42)

In [137]:
df = pd.read_csv('/Users/zarrina/Desktop/DS-Unit-4-Sprint-3-Neural-Networks/module4-Hyperparameter-Tuning/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [138]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 1.1+ MB


In [139]:
df = df.drop(['customerID','InternetService','gender'], axis=1).copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
cat_feat = ['MultipleLines', 'Contract', 'PaymentMethod']
bi_feat = ['PhoneService','Churn','Partner','Dependents','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']
for feat in bi_feat:
    df[feat] = df[feat] == 'Yes'
encoder = ce.OneHotEncoder(cols=cat_feat)    
df = encoder.fit_transform(df)
    
    
    

In [67]:
df.head()

,MultipleLines_1,MultipleLines_2,MultipleLines_3,MultipleLines_-1,Contract_1,Contract_2,Contract_3,Contract_-1,PaymentMethod_1,PaymentMethod_2,...,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,1,0,0,0,1,0,0,0,1,0,...,False,True,False,False,False,False,True,29.85,29.85,False
1,0,1,0,0,0,1,0,0,0,1,...,True,False,True,False,False,False,False,56.95,1889.50,False
2,0,1,0,0,1,0,0,0,0,1,...,True,True,False,False,False,False,True,53.85,108.15,True
3,1,0,0,0,0,1,0,0,0,0,...,True,False,True,True,False,False,False,42.30,1840.75,False
4,0,1,0,0,1,0,0,0,1,0,...,False,False,False,False,False,False,True,70.70,151.65,True


In [140]:
df['TotalCharges'].fillna(0, inplace=True)

In [141]:
def separate_features_label_values(df, target, scaler = StandardScaler()):
    X, y = df[df.columns[df.columns != target]], df[df.columns[df.columns == target]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X = scaler.transform(X)
    print(X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    return X, y, X_train, X_test, y_train, y_test
X, y, X_train, X_test, y_train, y_test = separate_features_label_values(df, 'Churn', scaler = StandardScaler())


(7043, 27) (7043, 1) (5634, 27) (1409, 27) (5634, 1) (1409, 1)


In [162]:
# model 
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Dense, Dropout

def create_model(lr=0.05,
                 activation='relu',                 
                 input_shape=(X_train.shape[1],),
                 optimizer=SGD,
                 relu_alpha = 0.003,
                 dropout_rate = 0.2,
                weight_initializer='random_normal'):
    
    # initialize a model
    model = Sequential()
    
    # add input layer
    model.add(Dense(10, input_shape=input_shape, kernel_initializer=weight_initializer,))
    model.add(LeakyReLU(alpha=relu_alpha)) 
    model.add(Dropout(rate=dropout_rate))

    
    # add hidden layers
    model.add(Dense(10, kernel_initializer=weight_initializer,))
    model.add(LeakyReLU(alpha=relu_alpha)) 
    model.add(Dropout(rate=dropout_rate))
        
    model.add(Dense(10, kernel_initializer=weight_initializer,))
    model.add(LeakyReLU(alpha=relu_alpha)) 
    model.add(Dropout(rate=dropout_rate))
    
    model.add(Dense(8, kernel_initializer=weight_initializer,))
    model.add(LeakyReLU(alpha=relu_alpha)) 
    model.add(Dropout(rate=dropout_rate))

    
    # add final output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # optimizer
    optimizer=optimizer(lr=lr)
    
    # compile model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
              
    return model
# sigmod = 0.71
# LeakyRelu = 0.75
# final LeackyRelu = 0.49

In [151]:
# baseline model
epochs = 10
batch_size = 100

model = KerasClassifier(build_fn=create_model, 
                               epochs=epochs,
                               batch_size=batch_size,
                               verbose=1)
kfold = StratifiedKFold(n_splits=3, random_state=42)
results = cross_val_score(model, X, y, cv=kfold)
print(f"K-fold Cross-Val Results - Mean: {results.mean():.2f} StDev: {results.std():.2f} MSE")

### Batch size

In [149]:
# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Epoch 1/20
4695/4695 [==============================] - 8s 2ms/step - loss: 0.5387 - acc: 0.7248
Epoch 2/20
4695/4695 [==============================] - 2s 371us/step - loss: 0.5019 - acc: 0.7329
Epoch 3/20
4695/4695 [==============================] - 2s 397us/step - loss: 0.4943 - acc: 0.7333
Epoch 4/20
4695/4695 [==============================] - 2s 396us/step - loss: 0.4845 - acc: 0.7333
Epoch 5/20
4695/4695 [==============================] - 2s 385us/step - loss: 0.4731 - acc: 0.7333
Epoch 6/20
4695/4695 [==============================] - 2s 407us/step - loss: 0.4707 - acc: 0.7544
Epoch 7/20
4695/4695 [==============================] - 2s 417us/step - loss: 0.4719 - acc: 0.7397
Epoch 8/20
4695/4695 [==============================] - 2s 424us/step - loss: 0.4727 - acc: 0.7410
Epoch 9/20
4695/4695 [==============================] - 2s 437us/step - loss: 0.4717 - acc: 0.7625
Epoch 10/20
4695/4695 [==============================] - 2s 431us/step - loss: 0.4663 - acc: 0.7796
Epoch 11/20

4695/4695 [==============================] - 1s 212us/step - loss: 0.4405 - acc: 0.7827
Epoch 20/20
4695/4695 [==============================] - 1s 124us/step
Epoch 1/20
4695/4695 [==============================] - 6s 1ms/step - loss: 0.5503 - acc: 0.7310
Epoch 2/20
4695/4695 [==============================] - 1s 195us/step - loss: 0.5128 - acc: 0.7363
Epoch 3/20
4695/4695 [==============================] - 1s 192us/step - loss: 0.4925 - acc: 0.7472
Epoch 4/20
4695/4695 [==============================] - 1s 195us/step - loss: 0.4905 - acc: 0.7555
Epoch 5/20
4695/4695 [==============================] - 1s 193us/step - loss: 0.4785 - acc: 0.7574
Epoch 6/20
4695/4695 [==============================] - 1s 193us/step - loss: 0.4688 - acc: 0.7604
Epoch 7/20
4695/4695 [==============================] - 1s 218us/step - loss: 0.4628 - acc: 0.7668
Epoch 8/20
4695/4695 [==============================] - 1s 203us/step - loss: 0.4655 - acc: 0.7736
Epoch 9/20
4695/4695 [=============================

4695/4695 [==============================] - 1s 107us/step - loss: 0.4665 - acc: 0.7463
Epoch 17/20
4695/4695 [==============================] - 0s 106us/step - loss: 0.4596 - acc: 0.7593
Epoch 18/20
4695/4695 [==============================] - 1s 107us/step - loss: 0.4536 - acc: 0.7700
Epoch 19/20
4695/4695 [==============================] - 1s 108us/step - loss: 0.4617 - acc: 0.7810
Epoch 20/20
4695/4695 [==============================] - 0s 62us/step
Epoch 1/20
4696/4696 [==============================] - 5s 1ms/step - loss: 0.5761 - acc: 0.7225
Epoch 2/20
4696/4696 [==============================] - 1s 118us/step - loss: 0.5271 - acc: 0.7349
Epoch 3/20
4696/4696 [==============================] - 1s 114us/step - loss: 0.5105 - acc: 0.7400
Epoch 4/20
4696/4696 [==============================] - 1s 114us/step - loss: 0.4987 - acc: 0.7460
Epoch 5/20
4696/4696 [==============================] - 1s 111us/step - loss: 0.4841 - acc: 0.7487
Epoch 6/20
4696/4696 [===========================

4696/4696 [==============================] - 0s 76us/step - loss: 0.4740 - acc: 0.7374
Epoch 15/20
4696/4696 [==============================] - 0s 78us/step - loss: 0.4653 - acc: 0.7376
Epoch 16/20
4696/4696 [==============================] - 0s 73us/step - loss: 0.4666 - acc: 0.7379
Epoch 17/20
4696/4696 [==============================] - 0s 74us/step - loss: 0.4611 - acc: 0.7379
Epoch 18/20
4696/4696 [==============================] - 0s 73us/step - loss: 0.4625 - acc: 0.7379
Epoch 19/20
4696/4696 [==============================] - 0s 74us/step - loss: 0.4576 - acc: 0.7376
Epoch 20/20
4696/4696 [==============================] - 0s 48us/step
Epoch 1/20
4695/4695 [==============================] - 6s 1ms/step - loss: 0.6210 - acc: 0.7022
Epoch 2/20
4695/4695 [==============================] - 0s 59us/step - loss: 0.5812 - acc: 0.7316
Epoch 3/20
4695/4695 [==============================] - 0s 62us/step - loss: 0.5712 - acc: 0.7335
Epoch 4/20
4695/4695 [==============================] -

4695/4695 [==============================] - 0s 50us/step - loss: 0.5138 - acc: 0.7495
Epoch 13/20
4695/4695 [==============================] - 0s 50us/step - loss: 0.5018 - acc: 0.7561
Epoch 14/20
4695/4695 [==============================] - 0s 52us/step - loss: 0.4960 - acc: 0.7572
Epoch 15/20
4695/4695 [==============================] - 0s 49us/step - loss: 0.4881 - acc: 0.7649
Epoch 16/20
4695/4695 [==============================] - 0s 51us/step - loss: 0.4945 - acc: 0.7617
Epoch 17/20
4695/4695 [==============================] - 0s 49us/step - loss: 0.4852 - acc: 0.7636
Epoch 18/20
4695/4695 [==============================] - 0s 54us/step - loss: 0.4906 - acc: 0.7608
Epoch 19/20
4695/4695 [==============================] - 0s 56us/step - loss: 0.4865 - acc: 0.7649
Epoch 20/20
4695/4695 [==============================] - 0s 34us/step
Epoch 1/20
4695/4695 [==============================] - 6s 1ms/step - loss: 0.6032 - acc: 0.7165
Epoch 2/20
4695/4695 [==============================]

### Epochs

In [150]:
# define the grid search parameters
param_grid = {'batch_size': [ 20, 40, 100],
              'epochs': [20, 30]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Epoch 1/20
4695/4695 [==============================] - 10s 2ms/step - loss: 0.5858 - acc: 0.7293
Epoch 2/20
4695/4695 [==============================] - 1s 249us/step - loss: 0.5429 - acc: 0.7333
Epoch 3/20
4695/4695 [==============================] - 1s 243us/step - loss: 0.5200 - acc: 0.7325
Epoch 4/20
4695/4695 [==============================] - 1s 239us/step - loss: 0.5049 - acc: 0.7331
Epoch 5/20
4695/4695 [==============================] - 1s 259us/step - loss: 0.4924 - acc: 0.7329
Epoch 6/20
4695/4695 [==============================] - 1s 246us/step - loss: 0.4913 - acc: 0.7327
Epoch 7/20
4695/4695 [==============================] - 1s 241us/step - loss: 0.4818 - acc: 0.7389
Epoch 8/20
4695/4695 [==============================] - 1s 271us/step - loss: 0.4793 - acc: 0.7542
Epoch 9/20
4695/4695 [==============================] - 2s 327us/step - loss: 0.4715 - acc: 0.7644
Epoch 10/20
4695/4695 [==============================] - 1s 243us/step - loss: 0.4674 - acc: 0.7629
Epoch 11/2

4695/4695 [==============================] - 2s 328us/step - loss: 0.4485 - acc: 0.7810
Epoch 20/30
4695/4695 [==============================] - 2s 415us/step - loss: 0.4552 - acc: 0.7776
Epoch 21/30
4695/4695 [==============================] - 1s 312us/step - loss: 0.4550 - acc: 0.7759
Epoch 22/30
4695/4695 [==============================] - 1s 286us/step - loss: 0.4513 - acc: 0.7845
Epoch 23/30
4695/4695 [==============================] - 1s 292us/step - loss: 0.4517 - acc: 0.7695
Epoch 24/30
4695/4695 [==============================] - 1s 276us/step - loss: 0.4500 - acc: 0.7781
Epoch 25/30
4695/4695 [==============================] - 1s 272us/step - loss: 0.4467 - acc: 0.7855
Epoch 26/30
4695/4695 [==============================] - 1s 276us/step - loss: 0.4476 - acc: 0.7791
Epoch 27/30
4695/4695 [==============================] - 1s 274us/step - loss: 0.4538 - acc: 0.7781
Epoch 28/30
4695/4695 [==============================] - 1s 296us/step - loss: 0.4453 - acc: 0.7902
Epoch 29/30


4695/4695 [==============================] - 1s 153us/step - loss: 0.5013 - acc: 0.7685
Epoch 8/20
4695/4695 [==============================] - 1s 221us/step - loss: 0.5025 - acc: 0.7604
Epoch 9/20
4695/4695 [==============================] - 1s 185us/step - loss: 0.4921 - acc: 0.7708
Epoch 10/20
4695/4695 [==============================] - 1s 138us/step - loss: 0.4897 - acc: 0.7732
Epoch 11/20
4695/4695 [==============================] - 1s 140us/step - loss: 0.4849 - acc: 0.7729
Epoch 12/20
4695/4695 [==============================] - 1s 138us/step - loss: 0.4859 - acc: 0.7687
Epoch 13/20
4695/4695 [==============================] - 1s 140us/step - loss: 0.4842 - acc: 0.7661
Epoch 14/20
4695/4695 [==============================] - 1s 147us/step - loss: 0.4721 - acc: 0.7800
Epoch 15/20
4695/4695 [==============================] - 1s 140us/step - loss: 0.4803 - acc: 0.7670
Epoch 16/20
4695/4695 [==============================] - 1s 138us/step - loss: 0.4734 - acc: 0.7723
Epoch 17/20
46

4695/4695 [==============================] - 1s 158us/step - loss: 0.4532 - acc: 0.7787
Epoch 26/30
4695/4695 [==============================] - 1s 161us/step - loss: 0.4564 - acc: 0.7800
Epoch 27/30
4695/4695 [==============================] - 1s 147us/step - loss: 0.4577 - acc: 0.7740
Epoch 28/30
4695/4695 [==============================] - 1s 155us/step - loss: 0.4518 - acc: 0.7793
Epoch 29/30
4695/4695 [==============================] - 1s 153us/step - loss: 0.4530 - acc: 0.7736
Epoch 30/30
4695/4695 [==============================] - 1s 141us/step
Epoch 1/30
4695/4695 [==============================] - 9s 2ms/step - loss: 0.5843 - acc: 0.7265
Epoch 2/30
4695/4695 [==============================] - 1s 195us/step - loss: 0.5392 - acc: 0.7327
Epoch 3/30
4695/4695 [==============================] - 1s 215us/step - loss: 0.5201 - acc: 0.7327
Epoch 4/30
4695/4695 [==============================] - 1s 184us/step - loss: 0.5096 - acc: 0.7327
Epoch 5/30
4695/4695 [=========================

4695/4695 [==============================] - 0s 72us/step - loss: 0.5051 - acc: 0.7340
Epoch 14/20
4695/4695 [==============================] - 0s 61us/step - loss: 0.5021 - acc: 0.7333
Epoch 15/20
4695/4695 [==============================] - 0s 60us/step - loss: 0.5084 - acc: 0.7323
Epoch 16/20
4695/4695 [==============================] - 0s 56us/step - loss: 0.4946 - acc: 0.7333
Epoch 17/20
4695/4695 [==============================] - 0s 58us/step - loss: 0.5076 - acc: 0.7340
Epoch 18/20
4695/4695 [==============================] - 0s 60us/step - loss: 0.4927 - acc: 0.7333
Epoch 19/20
4695/4695 [==============================] - 0s 60us/step - loss: 0.4991 - acc: 0.7344
Epoch 20/20
4695/4695 [==============================] - 0s 45us/step
Epoch 1/20
4695/4695 [==============================] - 8s 2ms/step - loss: 0.6366 - acc: 0.6686
Epoch 2/20
4695/4695 [==============================] - 0s 61us/step - loss: 0.5708 - acc: 0.7318
Epoch 3/20
4695/4695 [==============================] 

4695/4695 [==============================] - 8s 2ms/step - loss: 0.6149 - acc: 0.7171
Epoch 2/30
4695/4695 [==============================] - 0s 82us/step - loss: 0.5681 - acc: 0.7282
Epoch 3/30
4695/4695 [==============================] - 0s 67us/step - loss: 0.5641 - acc: 0.7306
Epoch 4/30
4695/4695 [==============================] - 0s 62us/step - loss: 0.5502 - acc: 0.7308
Epoch 5/30
4695/4695 [==============================] - 0s 81us/step - loss: 0.5489 - acc: 0.7316
Epoch 6/30
4695/4695 [==============================] - 0s 74us/step - loss: 0.5372 - acc: 0.7325
Epoch 7/30
4695/4695 [==============================] - 0s 59us/step - loss: 0.5322 - acc: 0.7325
Epoch 8/30
4695/4695 [==============================] - 0s 64us/step - loss: 0.5222 - acc: 0.7325
Epoch 9/30
4695/4695 [==============================] - 0s 69us/step - loss: 0.5186 - acc: 0.7359
Epoch 10/30
4695/4695 [==============================] - 0s 61us/step - loss: 0.5213 - acc: 0.7391
Epoch 11/30
4695/4695 [========

### Optimizer and learning rate

In [164]:
# create model,
# the number of epochs should be at least 20, as resuts show, but 
# my computer is getting really slow now, so the number of epochs is 7 

from keras.optimizers import SGD, Adam, Nadam

model = KerasClassifier(build_fn=create_model, 
                               epochs=7,
                               batch_size=100,
                               verbose=1)

# define the grid search parameters
param_grid = {'optimizer': [Adam, SGD],
              'lr': [.05, .03]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")


Epoch 1/7
4695/4695 [==============================] - 27s 6ms/step - loss: 0.5359 - acc: 0.7436
Epoch 2/7
4695/4695 [==============================] - 1s 112us/step - loss: 0.4780 - acc: 0.7638
Epoch 3/7
4695/4695 [==============================] - 1s 114us/step - loss: 0.4696 - acc: 0.7661
Epoch 4/7
4695/4695 [==============================] - 1s 113us/step - loss: 0.4696 - acc: 0.7659
Epoch 5/7
4695/4695 [==============================] - 1s 116us/step - loss: 0.4656 - acc: 0.7783
Epoch 6/7
4695/4695 [==============================] - 1s 148us/step - loss: 0.4779 - acc: 0.7587
Epoch 7/7
4695/4695 [==============================] - 0s 92us/step
Epoch 1/7
4695/4695 [==============================] - 27s 6ms/step - loss: 0.5259 - acc: 0.7237
Epoch 2/7
4695/4695 [==============================] - 1s 173us/step - loss: 0.5000 - acc: 0.7355
Epoch 3/7
4695/4695 [==============================] - 1s 130us/step - loss: 0.4722 - acc: 0.7557
Epoch 4/7
4695/4695 [==============================]

### Dropout Regularization

In [165]:
# create model
model = KerasClassifier(build_fn=create_model, 
                               epochs=7,
                               batch_size=100,
                               verbose=1)

# define the grid search parameters
param_grid = {'weight_initializer': ['random_normal'],
             'optimizer' : [Adam],
             'lr' : [0.03],
             'dropout_rate' : [0.0, 0.2, 0.3]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/7
3756/3756 [==============================] - 38s 10ms/step - loss: 0.5111 - acc: 0.7351
Epoch 2/7
3756/3756 [==============================] - 1s 143us/step - loss: 0.4388 - acc: 0.7894
Epoch 3/7
3756/3756 [==============================] - 0s 120us/step - loss: 0.4369 - acc: 0.7849
Epoch 4/7
3756/3756 [==============================] - 1s 149us/step - loss: 0.4336 - acc: 0.7854
Epoch 5/7
3756/3756 [==============================] - 0s 118us/step - loss: 0.4316 - acc: 0.7886
Epoch 6/7
3756/3756 [==============================] - 0s 130us/step - loss: 0.4257 - acc: 0.7910
Epoch 7/7
3756/3756 [==============================] - 0s 103us/step
Epoch 1/7
3756/3756 [==============================] - 18s 5ms/step - loss: 0.4981 - acc: 0.7417
Epoch 2/7
3756/3756 [==============================] - 0s 124us/step - loss: 0.4414 - acc: 0.7540
Epoch 3/7
3756/3756 [==============================] - 0s 115us/step - loss: 0.4247 - acc: 0.7961
Epoch 4/7
3756/3756 [=============================

For batches of 100 observations and 7 epochs, the best results were:
 Best: 0.7978345692940625 using {'dropout_rate': 0.0, 'lr': 0.03, 'optimizer': <class 'keras.optimizers.Adam'>, 'weight_initializer': 'random_normal'}

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?